In [1]:
from dotenv import load_dotenv
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import torch
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from evaluate import evaluator
import torch.cuda
import os
from huggingface_hub import InferenceClient


2024-03-03 11:38:07.002970: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-03 11:38:07.003056: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-03 11:38:07.006913: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-03 11:38:07.628954: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-03 11:38:09.965063: W tensorflow/compiler/

In [2]:
load_dotenv()
ACCESS_TOKEN = os.environ["HUGGING_FACES_TOKEN"]

In [3]:
client = InferenceClient(token=ACCESS_TOKEN)

In [6]:
echr = load_dataset("ecthr_cases",  "violation-prediction")
train_dataset, val_dataset, test_dataset = echr['train'], echr['validation'], echr['test']
train_dataset, val_dataset, test_dataset = [dataset.map( lambda examples: {"text": " ".join(examples["facts"])}) for dataset in [train_dataset, val_dataset, test_dataset]]
train_dataset, val_dataset, test_dataset = [dataset.map( lambda examples: {'labels' :list(1 if examples['labels'][i] else 0 for i in range(len(examples['labels'])))}, batched=True) for dataset in [train_dataset, val_dataset, test_dataset]]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [8]:
prompt = f"""Classify the text into VIOLATED, NON_VIOLATED about the result of the case. 
Text: {test_dataset[0]['text']}.
Result:
"""
print(prompt)

Classify the text into VIOLATED, NON_VIOLATED about the result of the case. 
Text: 5.  The applicant is a journalist for DN.no, a Norwegian Internet-based version of the newspaper Dagens Næringsliv (“DN”), published by the company DN Nye Medier AS. 6.  On 23 June 2010 Mr X was indicted for market manipulation and insider trading under the 1997 Act on the Trade of Financial Assets (verdipapirhandelloven). He was accused of having requested Mr Y, an attorney, to draft a letter concerning the Norwegian Oil Company (“DNO”), a limited liability company quoted on the stock exchange. The letter, addressed to a trustee company representing the interests of bond holders in DNO (“the bond trustee company”), gave the impression that it had been written on behalf of a number of bond holders who were seriously concerned about the company’s liquidity, finances and future. In fact, it had been written only on Mr X’s behalf. He had owned only one bond, which he had acquired the same day as he had aske

In [6]:
client.text_generation(prompt, model="l-yohai/bigbird-roberta-base-mnli")

HfHubHTTPError: 500 Server Error: Internal Server Error for url: https://api-inference.huggingface.co/models/l-yohai/bigbird-roberta-base-mnli (Request ID: 1-UYHq-Nvrh9D4N_NaFAz)

unknown error